In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [2]:
# Import necessary libraries
from training import SummarizationDataLoader, Summarizer

import torch
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer

In [3]:
# Define los callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='../checkpoints/',  # Ruta donde se guardarán los modelos
    filename='best-checkpoint',
    save_top_k=1,
    mode='min'
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

In [4]:
data_module = SummarizationDataLoader()
model = Summarizer()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
# Inicializa el entrenador con los callbacks
trainer = Trainer(
    max_epochs=10,
    callbacks=[checkpoint_callback, early_stopping_callback],
    devices=1 if torch.cuda.is_available() else 1,
    accelerator="gpu" if torch.cuda.is_available() else "cpu"
)

# Entrena el modelo
trainer.fit(model, datamodule=data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Martin\miniconda3\envs\text_summarization_with_T5ForConditionalGeneration\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
c:\Users\Martin\miniconda3\envs\text_summarization_with_T5ForConditionalGeneration\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_d

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Martin\miniconda3\envs\text_summarization_with_T5ForConditionalGeneration\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
c:\Users\Martin\miniconda3\envs\text_summarization_with_T5ForConditionalGeneration\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

IndexError: piece id is out of range.

In [ ]:
# Para cargar el mejor modelo guardado
best_model_path = checkpoint_callback.best_model_path

# Cargar el modelo desde el checkpoint
model = Summarizer.load_from_checkpoint(best_model_path)

print(f"El mejor modelo se ha guardado en: {best_model_path}")

In [ ]:
# Asegúrate de que el modelo esté en modo de evaluación
model.eval()

# Ejemplo de uso del modelo para hacer predicciones
# Supongamos que tienes una función de preprocesamiento y un texto de entrada
def preprocess(text):
    # Implementa la lógica de preprocesamiento aquí
    return text

input_text = "Este es un texto de ejemplo para resumir."
preprocessed_text = preprocess(input_text)

# Convertir el texto preprocesado a un tensor
input_tensor = torch.tensor(preprocessed_text)

# Hacer una predicción
with torch.no_grad():
    summary = model(input_tensor)

print("Resumen:", summary)